# Create summary and evaluation of different runs

In [56]:
# !rm -rf snellius_results
!rsync -r snellius:~/uvadlc_practicals_2023/assignment2/part2/results_zs snellius_results
!rsync -r snellius:~/uvadlc_practicals_2023/assignment2/part2/results_vp snellius_results

In [57]:
import json
from pathlib import Path
import pandas as pd

In [58]:
snellius_results = Path("snellius_results")

def load_jsons_to_df(json_dir):
    json_data = []
    for js_path in json_dir.glob("*.json"):
        with open(js_path) as f: 
            js = json.load(f)
            json_data.append(js)
    return pd.DataFrame(json_data)

vp_dp_df = load_jsons_to_df(snellius_results / "results_vp")
zs_df = load_jsons_to_df(snellius_results / "results_zs")

In [59]:
zs_df

,dataset,set,accuracy
0,cifar100,test,63.079071
1,cifar10,train,88.721609
2,cifar10,test,88.897765
3,cifar100,train,63.581651


In [60]:
vp_dp_df

,print_freq,print_tqdm_interval,save_freq,batch_size,num_workers,epochs,max_batches,square_size,optim,learning_rate,...,resume_best,evaluate,gpu,use_wandb,device,model_folder,start_epoch,top1_val_acc,top1_test_acc,best_epoch
0,10,60.0,50,128,16,20,0,8,sgd,40,...,False,False,None,False,cuda,./save/models/deep_prompt_padding_30_cifar10_c...,20,79.98,79.24,20.0
1,10,60.0,50,128,16,20,0,8,sgd,40,...,True,True,None,False,cuda,./save/models/visual_prompt_fixed_patch_1_cifa...,7,NaN,60.35,7.0
2,10,60.0,50,128,16,20,0,8,sgd,40,...,False,False,None,False,cuda,./save/models/deep_prompt_padding_30_cifar10_c...,20,53.97,52.73,NaN
3,10,60.0,50,128,16,20,0,8,sgd,40,...,False,False,None,False,cuda,./save/models/deep_prompt_padding_30_cifar10_c...,1,76.75,75.83,NaN
4,10,60.0,50,128,16,20,0,8,sgd,40,...,False,False,None,False,cuda,./save/models/visual_prompt_padding_1_cifar100...,4,67.31,66.99,4.0
5,10,60.0,50,128,16,20,0,8,sgd,40,...,True,True,None,False,cuda,./save/models/visual_prompt_padding_1_cifar10_...,4,NaN,87.54,4.0
6,10,60.0,50,128,16,20,0,8,sgd,40,...,True,True,None,False,cuda,./save/models/deep_prompt_padding_30_cifar10_c...,1,NaN,75.05,NaN
7,10,60.0,50,128,16,20,0,8,sgd,40,...,True,True,None,False,cuda,./save/models/deep_prompt_padding_30_cifar10_c...,20,NaN,75.07,20.0
8,10,60.0,50,128,16,20,0,8,sgd,40,...,True,True,None,False,cuda,./save/models/deep_prompt_padding_30_cifar100_...,20,NaN,46.10,20.0
9,10,60.0,50,128,16,20,0,8,sgd,40,...,False,False,None,False,cuda,./save/models/deep_prompt_padding_30_cifar100_...,20,54.83,54.78,20.0


In [61]:
vp_cols = ["dataset", "prompt_num", "method", "prompt_size", "test_noise", "top1_test_acc", "best_epoch"]
dp_cols = ["dataset", "prompt_num", "injection_layer", "prompt_size", "test_noise", "top1_test_acc", "best_epoch"]

In [62]:
vp_df = vp_dp_df.query("prompt_type == 'visual_prompt'")[vp_cols].sort_values(["dataset", "method", "test_noise"])
vp_df

,dataset,prompt_num,method,prompt_size,test_noise,top1_test_acc,best_epoch
17,cifar10,4,fixed_patch,1,False,89.25,NaN
12,cifar10,4,fixed_patch,1,True,87.65,NaN
18,cifar10,4,padding,1,False,89.89,NaN
5,cifar10,4,padding,1,True,87.54,4.0
10,cifar100,4,fixed_patch,1,False,64.30,7.0
1,cifar100,4,fixed_patch,1,True,60.35,7.0
4,cifar100,4,padding,1,False,66.99,4.0
19,cifar100,4,padding,1,True,60.82,4.0


In [63]:
dp_df = vp_dp_df.query("prompt_type == 'deep_prompt'")[dp_cols].sort_values(["dataset", "injection_layer", "test_noise"])
dp_df

,dataset,prompt_num,injection_layer,prompt_size,test_noise,top1_test_acc,best_epoch
3,cifar10,4,0,30,False,75.83,NaN
6,cifar10,4,0,30,True,75.05,NaN
2,cifar10,4,3,30,False,52.73,NaN
16,cifar10,4,3,30,True,49.85,20.0
0,cifar10,4,7,30,False,79.24,20.0
7,cifar10,4,7,30,True,75.07,20.0
13,cifar100,4,0,30,False,49.14,20.0
15,cifar100,4,0,30,True,47.38,20.0
11,cifar100,4,3,30,False,53.59,20.0
8,cifar100,4,3,30,True,46.10,20.0
